<a href="https://colab.research.google.com/github/yj2213/Toy_Project/blob/main/Toyproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch import nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

!pip install torchinfo
import torchinfo
from torchinfo import summary

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# Raw dataset
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_labels = {}

        # Create a mapping of class labels to integers
        self.class_labels = {}
        class_idx = 0

        # Iterate over sub-directories
        for class_dir in os.listdir(self.root_dir):
            class_dir_path = os.path.join(self.root_dir, class_dir)
            if os.path.isdir(class_dir_path):
                self.class_labels[class_dir] = class_idx
                class_idx += 1

                # Iterate over images in the sub-directory
                for img_filename in os.listdir(class_dir_path):
                    if img_filename.endswith(".jpg"):
                        img_path = os.path.join(class_dir_path, img_filename)
                        self.images.append(img_path)
                        self.labels.append(self.class_labels[class_dir])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]

        # 그레이스케일 이미지가 있는지 확인. 있으면 -> 색상 채널 추가
        if image.mode == "L":
            image = Image.merge("RGB", (image, image, image))
        #"transform이 있는지 확인. 있으면 -> transform(image)"
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
! kaggle datasets download -d gpiosenka/sports-classification

Dataset URL: https://www.kaggle.com/datasets/gpiosenka/sports-classification
License(s): CC0-1.0
 98% 415M/424M [00:04<00:00, 98.7MB/s]
100% 424M/424M [00:04<00:00, 94.6MB/s]


In [ ]:
import zipfile

zip_file = './sports-classification.zip'
path_to_extract = './kaggle/input/sports-classification/'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(path_to_extract)

import os
os.remove(zip_file)

In [ ]:
def model_dataloder(weights, transform):
    """
    Will return 3 pytorch datalaoder
    """
    weights = weights

    data_folder = "./kaggle/input/sports-classification"

    train_folder = data_folder + "/train"
    val_folder = data_folder + "/valid"
    test_folder = data_folder + "/test"

    # pytorch dataset
    train_dataset = ImageDataset(train_folder, transform = transform)
    val_dataset = ImageDataset(val_folder, transform = transform)
    test_dataset = ImageDataset(test_folder, transform = transform)

    # pytorch dataloader
    train_dataloader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)
    val_dataloader = DataLoader(dataset = val_dataset, batch_size = 32, shuffle = False)
    test_dataloader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = False)

    return train_dataloader, val_dataloader, test_dataloader

In [ ]:
# Train -> train_loss, train_acc
def train (model, dataloader, loss_fn, optimizer, device):
    train_loss, train_acc = 0, 0

    model.to(device)
    model.train()

    for batch, (x, y) in enumerate (dataloader):
        x, y = x.to(device), y.to(device)

        train_pred = model(x)

        loss = loss_fn(train_pred, y)
        train_loss = train_loss + loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_pred_label = torch.argmax(torch.softmax(train_pred, dim = 1), dim = 1)
        train_acc = train_acc + (train_pred_label == y).sum().item() / len(train_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

In [ ]:
# Val -> val_loss, val_acc
def val (model, dataloader, loss_fn, device):
    val_loss, val_acc = 0, 0

    model.to(device)
    model.eval()

    with torch.inference_mode():
        for batch, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            val_pred = model(x)

            loss = loss_fn(val_pred, y)
            val_loss = val_loss + loss.item()

            val_pred_label = torch.argmax(torch.softmax(val_pred, dim = 1), dim = 1)
            val_acc = val_acc + (val_pred_label == y).sum().item() / len(val_pred)

        val_loss = val_loss / len(dataloader)
        val_acc = val_acc / len(dataloader)

        return val_loss, val_acc

In [ ]:
import copy
from tqdm import tqdm
def training_loop(model, train_dataloader, val_dataloader, device, epochs, patience):
    # empty dict to store results
    results = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    # hardcoded loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    best_val_loss = float("inf")
    num_no_improvement = 0

    # loop through epochs
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Training phase with tqdm progress bar
        train_loader = tqdm(train_dataloader, desc="Training", leave=False)
        train_loss, train_acc = train(
            model=model,
            dataloader=train_loader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        # Validation phase with tqdm progress bar
        val_loader = tqdm(val_dataloader, desc="Validation", leave=False)
        val_loss, val_acc = val(
            model=model,
            dataloader=val_loader,
            loss_fn=loss_fn,
            device=device
        )

        # print results for each epoch
        print(f"Train loss: {train_loss:.4f} | Train accuracy: {(train_acc * 100):.3f}%")
        print(f"Val loss: {val_loss:.4f} | Val accuracy: {(val_acc * 100):.3f}%")

        # record results for each epoch
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["val_loss"].append(val_loss)
        results["val_acc"].append(val_acc)

        # check for early stopping condition
        mean_val_loss = np.mean(results["val_loss"])
        best_val_loss = float("inf")
        num_no_improvement = 0
        if np.mean(mean_val_loss > best_val_loss):
            best_val_loss = mean_val_loss

            model_state_dict = model.state_dict()
            best_model = copy.deepcopy(model)
            best_model.load_state_dict(model_state_dict)
        else:
            num_no_improvement +=1

        if num_no_improvement == patience:
            break

    # Plot results after early stopping
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Loss")
    plt.plot(results["train_loss"], label="Train loss")
    plt.plot(results["val_loss"], label="Val loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.title("Accuracy")
    plt.plot(results["train_acc"], label="Train accuracy")
    plt.plot(results["val_acc"], label="Val accuracy")
    plt.legend()

    plt.show()

    return results


RESNET50

In [ ]:
resnet_weight = torchvision.models.ResNet50_Weights.DEFAULT

resnet_model = torchvision.models.resnet50(weights = resnet_weight)

for param in resnet_model.parameters():
    param.requires_grad = False

In [ ]:
# Custom output layer
# resnet_model.fc

custom_fc = nn.Sequential(
    nn.ReLU(),
    nn.Dropout(p = 0.5),
    nn.Linear(1000, 100))

resnet_model.fc = nn.Sequential(
    resnet_model.fc,
    custom_fc
)

In [ ]:
# Check model info
summary(resnet_model, input_size = (1, 3, 244, 244), col_names = ["output_size", "num_params", "trainable"], col_width = 15)

Layer (type:depth-idx)                   Output Shape    Param #         Trainable
ResNet                                   [1, 100]        --              Partial
├─Conv2d: 1-1                            [1, 64, 122, 122] (9,408)         False
├─BatchNorm2d: 1-2                       [1, 64, 122, 122] (128)           False
├─ReLU: 1-3                              [1, 64, 122, 122] --              --
├─MaxPool2d: 1-4                         [1, 64, 61, 61] --              --
├─Sequential: 1-5                        [1, 256, 61, 61] --              False
│    └─Bottleneck: 2-1                   [1, 256, 61, 61] --              False
│    │    └─Conv2d: 3-1                  [1, 64, 61, 61] (4,096)         False
│    │    └─BatchNorm2d: 3-2             [1, 64, 61, 61] (128)           False
│    │    └─ReLU: 3-3                    [1, 64, 61, 61] --              --
│    │    └─Conv2d: 3-4                  [1, 64, 61, 61] (36,864)        False
│    │    └─BatchNorm2d: 3-5             [1, 64

Train models

In [ ]:
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
# hardcode :
# loss_fn -> CrossEntropyLoss
# optimizer -> Adam(lr = 0.0005)

Resnet50

In [ ]:
# Data augmentation
# resnet_weight.transforms()
resnet_transform = transforms.Compose([
    transforms.Resize(size = 232),
    transforms.ColorJitter(brightness = (0.8, 1.2)),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.RandomRotation(degrees = 15),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
])

resnet_train_dataloader, resnet_val_dataloader, resnet_test_dataloader = model_dataloder(weights = resnet_weight,
                                                                                         transform = resnet_transform
                                                                                        )

In [ ]:
# Actual training ResNet model
resnet_results = training_loop(model = resnet_model,
                               train_dataloader = resnet_train_dataloader,
                               val_dataloader = resnet_val_dataloader,
                               device = device,
                               epochs = 5,
                               patience = 5
                              )



Epoch 1/5


Train loss: 1.0806 | Train accuracy: 75.065%
Val loss: 0.7828 | Val accuracy: 81.641%


NameError: name 'best_model' is not defined

Evaluation

In [ ]:
# empty list store labels
predict_label_list = []
actual_label_list = []

# eval mode
resnet_model.eval()

for images, labels in resnet_test_dataloader:

    for label in labels:
        label = label.item()
        actual_label_list.append(label)

    for image in images:
        with torch.inference_mode():
            image = image.to(device)
            # add batch_size and device
            image = image.unsqueeze(dim = 0)
            # logits
            logits = resnet_model(image)
            # lables
            label = torch.argmax(logits).item()
            predict_label_list.append(label)

Accuracy score

In [ ]:
accuracy = accuracy_score(actual_label_list, predict_label_list)

print(f"resnet_model's accuracy: {accuracy*100}%")

resnet_model's accuracy: 87.0%
